In [1]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import pickle
import joblib
import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

from hyperopt.pyll.base import scope
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from kaggler.model import AutoLGB
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
import random

# 1. 문제 정의

# 2. 데이터 수집

## (1) 데이콘 기본 데이터

In [2]:
train = pd.read_csv('data/train.csv').drop(['index'], axis=1).fillna('NAN')
test = pd.read_csv('data/test.csv').drop(['index'], axis=1).fillna('NAN')
sample_submission = pd.read_csv('data/sample_submission.csv')

# 3. 탐색적 데이터 분석

# 4. 변수 조정

In [3]:
# train데이터와 test데이터 변수를 함께 조정하기 위해 병합
merge_data = pd.concat([train, test], axis = 0)

# DAYS_BIRTH
merge_data['DAYS_BIRTH_month']=np.floor((-merge_data['DAYS_BIRTH'])/30)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/30)/12).astype(int)*12)
merge_data['DAYS_BIRTH_week']=np.floor((-merge_data['DAYS_BIRTH'])/7)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/7)/4).astype(int)*4)

# DAYS_EMPLOYED
merge_data['DAYS_EMPLOYED_month']=np.floor((-merge_data['DAYS_EMPLOYED'])/30)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['DAYS_EMPLOYED_week']=np.floor((-merge_data['DAYS_EMPLOYED'])/7)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

# before_EMPLOYED
merge_data['before_EMPLOYED']=merge_data['DAYS_BIRTH']-merge_data['DAYS_EMPLOYED']
merge_data['before_EMPLOYED_month']=np.floor((-merge_data['before_EMPLOYED'])/30)-(
    (np.floor((-merge_data['before_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['before_EMPLOYED_week']=np.floor((-merge_data['before_EMPLOYED'])/7)-(
    (np.floor((-merge_data['before_EMPLOYED'])/7)/4).astype(int)*4)

# DAYS_BIRTH
merge_data['1new_1'] = merge_data['DAYS_BIRTH_month'] / merge_data['income_total']
merge_data['2new_1'] = merge_data['DAYS_BIRTH_week'] / merge_data['income_total']

# DAYS_EMPLOYED
merge_data['10new_1'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['income_total']
merge_data['11new_1'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['income_total']

# before_EMPLOYED
merge_data['12new_1'] = merge_data['before_EMPLOYED'] / merge_data['income_total']
merge_data['13new_1'] = merge_data['before_EMPLOYED_month'] / merge_data['income_total']
merge_data['14new_1'] = merge_data['before_EMPLOYED_week'] / merge_data['income_total']

# 총 수익을 가족 수로 나누기
merge_data['15new_1'] = merge_data['income_total'] / merge_data['family_size']

# 융합 삭제
#merge_data['3new_1'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['DAYS_BIRTH_month']
#merge_data['4new_1'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['DAYS_BIRTH_week']
#merge_data['5new_1'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['DAYS_BIRTH_month']
#merge_data['6new_1'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['DAYS_BIRTH_week']

#merge_data['7new_1'] =  merge_data['begin_month'] / merge_data['DAYS_BIRTH_month']
#merge_data['8new_1'] =  merge_data['begin_month'] / merge_data['DAYS_EMPLOYED_month']
#merge_data['9new_1'] =  merge_data['begin_month'] / merge_data['before_EMPLOYED_month']

merge_data['new_1'] = merge_data['child_num'] / merge_data['income_total']
merge_data['new_2'] = merge_data['family_size'] / merge_data['income_total']
merge_data['new_3'] = merge_data['DAYS_BIRTH'] / merge_data['income_total']
merge_data['new_4'] = merge_data['DAYS_EMPLOYED'] / merge_data['income_total']
#merge_data['new_5'] = merge_data['begin_month'] / merge_data['income_total']
merge_data['new_6'] =  merge_data['DAYS_EMPLOYED'] / merge_data['DAYS_BIRTH']

# 소득 skewed-data 처리
#merge_data['log1p_income_total'] = np.log1p(merge_data['income_total'])
# merge_data['log_income_total'] = np.log(merge_data['income_total'])
# merge_data['sqrt_income_total'] = np.sqrt(merge_data['income_total'])
# merge_data['boxcox_income_total'] = stats.boxcox(merge_data['income_total'])[0]

merge_data = merge_data.fillna(-999)
train = merge_data[merge_data['credit'] != -999]
test = merge_data[merge_data['credit'] == -999]
test.drop('credit', axis = 1, inplace = True)

train_cols = list(train.columns); train_cols.remove('credit'); train_cols.append('credit')
train = train[train_cols]

train = train[train['child_num']<=6].reset_index(drop=True) # 아이의 수가 7명 이상인 데이터 제거

## 인코딩

remove_features = [2, 4, 23]
train = train.drop(train.columns[remove_features], axis=1)
test = test.drop(test.columns[remove_features], axis=1)

In [4]:
train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()

In [5]:
object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [6]:
object_col = []
for col in train_oh.columns:
    if train_oh[col].dtype == 'object':
        object_col.append(col)
print(object_col)        
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train_oh.drop(object_col, axis=1, inplace=True)
train_oh = pd.concat([train_oh, train_onehot_df], axis=1)    

test_onehot_df = pd.DataFrame(enc.transform(test_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test_oh.drop(object_col, axis=1, inplace=True)
test_oh = pd.concat([test_oh, test_onehot_df], axis=1)

['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']


## Feature 하나씩 빼면서 성능 체크

변수 하나씩 제거하면서 성능 체크<br>
제거하여 성능이 좋게 나온 것들은 리스트에 따로 저장해두기

In [7]:
train_x = train_noh.drop(['credit'], axis=1)
train_y = train_noh['credit']
test_x = test_noh.copy()

In [8]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# cv = np.zeros((train_x.shape[0], 3))
# for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
#     x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
#     y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
#     lgbm = LGBMClassifier(n_estimators=1000, objective='multiclass')
#     lgbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None)
#     cv[val_idx] = lgbm.predict_proba(x_val)
# Initial_log_loss = log_loss(train_y, cv)
# print(f'Initial_multi_logloss: {Initial_log_loss}')

# remove_features = []
# for i in range(1, 2):
#     for j in combinations(list(range(0, train_x.shape[1])), i):
#         train_new_x = train_x.drop(train_x.columns[list(j)], axis=1)
        
#         kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
#         cv = np.zeros((train_new_x.shape[0], 3))
#         for n, (train_idx, val_idx) in enumerate(kfold.split(train_new_x, train_y)):
#             x_train, x_val = train_new_x.iloc[train_idx], train_new_x.iloc[val_idx]
#             y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
#             lgbm = LGBMClassifier(n_estimators=1000, objective='multiclass')
#             lgbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None)
#             cv[val_idx] = lgbm.predict_proba(x_val)
#         print(f'{j} multi_logloss: {log_loss(train_y, cv)}')
#         if log_loss(train_y, cv) < Initial_log_loss:
#             remove_features.append(list(j)[0])

## 저장한 변수 지우는 코드

In [ ]:
remove_features = [1, 3, 4, 8, 11, 32]
train = train.drop(train.columns[remove_features], axis=1)
test = test.drop(test.columns[remove_features], axis=1)

## 다시 인코딩

In [ ]:
train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()

In [ ]:
object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [ ]:
object_col = []
for col in train_oh.columns:
    if train_oh[col].dtype == 'object':
        object_col.append(col)
print(object_col)        
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train_oh.drop(object_col, axis=1, inplace=True)
train_oh = pd.concat([train_oh, train_onehot_df], axis=1)    

test_onehot_df = pd.DataFrame(enc.transform(test_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test_oh.drop(object_col, axis=1, inplace=True)
test_oh = pd.concat([test_oh, test_onehot_df], axis=1)

# 6. 모델 학습

In [ ]:
pred_dict = {}
pred_test_dict = {}

## (1) Lightgbm

In [ ]:
train_x = train_noh.drop(['credit'], axis=1)
train_y = train_noh['credit']
test_x = test_noh.copy()

### Parameter Tuning

In [ ]:
# SEED=0

# # Hyperopt의 metric함수를 StratifiedKFold(cv=5)로 구하기
# def score(params):
#     print("Training with params: ")
#     print(params)
    
#     kfold = StratifiedKFold(n_splits=5, random_state = SEED, shuffle = True)
#     cv = np.zeros((train_x.shape[0], 3))

#     for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
#         x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
#         y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

#         lgbmodel = LGBMClassifier(**params)

#         lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None) 
#         cv[val_idx, :] = lgbmodel.predict_proba(x_val)
#         #print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
#     print('multi_logloss:', log_loss(train_y, cv))
#     score = log_loss(train_y, cv)
#     return {'loss': score, 'status': STATUS_OK}

# # Hyperopt의 범위를 지정해주고 max_evals만큼 반복한 후 최적의 파라미터를 반환
# def optimize(random_state=SEED):
    
# #     param = {'objective':'multi:softprob', 'seed':SEED, 'num_class': 3, 'eval_metric':'mlogloss', 
# #          'eta': 0.01, 'min_child_weight': 3,
# #          'colsample_bytree': 0.3, 'colsample_bylevel': 0.6, 'subsample': 0.8
# #         }
#     space = {
#         #'learning_rate': hp.quniform('learning_rate', 0.004, 0.005, 0.001),
#         'learning_rate' : 0.004,
#         #'num_leaves': scope.int(hp.quniform('num_leaves', 1000, 1200, 50)),
#         'num_leaves' : 1000,
#         #'min_child_weight': hp.quniform('min_child_weight', 1, 3, 1),
#         'min_child_weight' : 2,
#         #'subsample': hp.quniform('subsample', 0.8, 1, 0.05),
#         'subsample' : 1,
#         #'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 0.61, 0.01),
#         'colsample_bytree' : 0.6,
#         #'reg_alpha': hp.quniform('lambda_l1', 0.9, 1, 0.01),
#         'reg_alpha' : 0.94,
#         #'reg_lambda': hp.quniform('lambda_l2', 0.8, 1, 0.02),
#         'reg_lambda' : 0.98,
#         'max_depth' : -1,
#         'n_estimators' : 5000,
#         'objective' : 'multiclass',
#         'num_class' : 3,
#         'seed': SEED,
#     }
#     # Use the fmin function from Hyperopt to find the best hyperparameters
#     best = fmin(score, space, algo=tpe.suggest, 
#                 # trials=trials, 
#                 max_evals=12)
#     return best

# best_hyperparams = optimize()
# print("The best hyperparameters are: ", "\n")
# print(best_hyperparams)

### 3 seeds x 10 folds

In [ ]:
lucky_seeds=[100,2019,91373] # Lucky seed 늘려가면서 하기
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True) # CV 늘려가면서 하기
    cv=np.zeros((train_x.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        lgbmodel = LGBMClassifier(learning_rate=0.004, objective='multiclass', n_estimators=10000, num_leaves=1000, 
                                  max_depth=-1, min_child_weight=2, colsample_bytree=0.6, reg_alpha=0.94, reg_lambda=0.98,
                                   n_jobs=-1, random_state=seed)

        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None) 
        #joblib.dump(lgbmodel, f'./pred_pkl/LGB_{n+1}_fold_{seed}_seed_lgb.pkl')

        # CROSS-VALIDATION , EVALUATE CV
        cv[val_idx,:] = lgbmodel.predict_proba(x_val)
        pred_test += lgbmodel.predict_proba(test_x) / 10 # CV 바꾸면 이 숫자도 똑같이 바꿔야함
    pred_dict['lgb'+str(seed)] = cv
    pred_test_dict['lgb'+str(seed)] = pred_test
        
    print('multi_logloss :', log_loss(train_y, cv))

lgbmodels_path = os.listdir('./pred_pkl/')
lgbmodels_list = [x for x in lgbmodels_path if x.endswith("lgb.pkl")]
assert len(lgbmodels_list) == 15
lgb_preds = np.zeros((test_x.shape[0], 3))

for m in lgbmodels_list:
    lgbmodel = joblib.load('./pred_pkl/'+m)
    lgb_preds_proba = lgbmodel.predict_proba(test)
    lgb_preds += lgb_preds_proba/15

## (2) XGBoost

원핫인코딩된 feature로 만들어주기 **꼭 밑에 코드 실행하고 XGBoost랑 Randomforest 돌리기!!**

In [ ]:
train_x = train_oh.drop(['credit'], axis=1)
train_y = train_oh['credit']
test_x = test_oh.copy()

### Parameter Tuning (hyperopt)

In [ ]:
# SEED=0

# # Hyperopt의 metric함수를 StratifiedKFold(cv=5)로 구하기
# def score(params):
#     print("Training with params: ")
#     print(params)
    
#     kfold = StratifiedKFold(n_splits=5, random_state = SEED, shuffle = True)
#     cv = np.zeros((train_x.shape[0], 3))

#     for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
#         x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
#         y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
#         dtrain = xgb.DMatrix(x_train, label=y_train)
#         dvalid = xgb.DMatrix(x_val, label=y_val)
#         watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

#         xgbmodel = xgb.train(params, dtrain, 100000, watchlist, early_stopping_rounds=30, verbose_eval=None)
#         cv[val_idx, :] = xgbmodel.predict(dvalid)
#         print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
#     print('multi_logloss:', log_loss(train_y, cv))
#     score = log_loss(train_y, cv)
#     return {'loss': score, 'status': STATUS_OK}

# # Hyperopt의 범위를 지정해주고 max_evals만큼 반복한 후 최적의 파라미터를 반환
# def optimize(random_state=SEED):
    
# #     param = {'objective':'multi:softprob', 'seed':SEED, 'num_class': 3, 'eval_metric':'mlogloss', 
# #          'eta': 0.01, 'min_child_weight': 3,
# #          'colsample_bytree': 0.3, 'colsample_bylevel': 0.6, 'subsample': 0.8
# #         }
#     space = {
#         #'eta': hp.quniform('eta', 0.006, 0.007, 0.001),
#         'eta' : 0.005,
#         #'max_depth':  hp.choice('max_depth', np.arange(20, 30, dtype=int)),
#         #'min_child_weight': hp.quniform('min_child_weight', 4, 5, 1),
#         'min_child_weight': 4,
#         #'subsample': hp.quniform('subsample', 0.72, 0.8, 0.02),
#         'subsample': 0.72,
#         #'gamma': hp.quniform('gamma', 0.6, 0.8, 0.05),
#         'gamma': 0.7,
#         #'colsample_bytree': hp.quniform('colsample_bytree', 0.3, 0.7, 0.05),
#         'colsample_bytree': 0.45,
#         #'colsample_bylevel': hp.quniform('colsample_bylevel', 0.1, 0.15, 0.01),
#         'colsample_bylevel': 0.15,
#         'max_depth' : 100,
#         'eval_metric': 'mlogloss',
#         'objective' : 'multi:softprob',
#         'num_class' : 3,
#         'seed': SEED,
#     }
#     # Use the fmin function from Hyperopt to find the best hyperparameters
#     best = fmin(score, space, algo=tpe.suggest, 
#                 # trials=trials, 
#                 max_evals=10)
#     return best

# best_hyperparams = optimize()
# print("The best hyperparameters are: ", "\n")
# print(best_hyperparams)

### 3 seeds x 10 folds

In [ ]:
lucky_seeds=[0, 100, 91373] # 늘려가면서
xgtest = xgb.DMatrix(test_x)
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True) # 늘려가면서
    cv = np.zeros((train.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        dtrain = xgb.DMatrix(x_train, label=y_train)
        dvalid = xgb.DMatrix(x_val, label=y_val)
        watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

        param = {'colsample_bylevel': 0.15, 'colsample_bytree': 0.45, 'eta': 0.005, 'eval_metric': 'mlogloss', 
         'gamma': 0.7, 'max_depth': 100, 'min_child_weight': 4, 'num_class': 3, 
         'objective': 'multi:softprob', 'seed': 0, 'subsample': 0.72}
        xgbmodel = xgb.train(param, dtrain, 100000, watchlist, early_stopping_rounds=30, verbose_eval=None)
        #joblib.dump(xgbmodel, f'./pred_pkl/XGB_{n+1}_fold_{seed}_seed_xgb.pkl')

        cv[val_idx, :] = xgbmodel.predict(dvalid)
        #print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
        pred_test += xgbmodel.predict(xgtest) / 10 # CV 바꾸면 이 숫자도 똑같이 바꿔야함
        
    pred_dict['xgb'+str(seed)] = cv
    pred_test_dict['xgb'+str(seed)] = pred_test
    print('multi_logloss:', log_loss(train_y, cv))

xgbmodels_path = os.listdir('./pred_pkl/')
xgbmodels_list = [x for x in xgbmodels_path if x.endswith("xgb.pkl")]
assert len(xgbmodels_list) == 15
xgb_preds = np.zeros((test_x.shape[0], 3))
xgtest = xgb.DMatrix(test_X)

for m in xgbmodels_list:
    xgbmodel = joblib.load('./pred_pkl/'+m)
    xgb_preds_proba = xgbmodel.predict_proba(xgtest
       xgb_preds += xgb_preds_proba/15

## (3) Random Forest

### GridSearchCV

params = {'max_depth': [55, 60, 65] # 튜닝할 파라미터 삽입
            }

rf_clf = RandomForestClassifier(random_state = 0, n_estimators = 1000, 
                                min_samples_leaf=2, min_samples_split=2,
                                criterion='entropy', n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 5, n_jobs = -1)
grid_cv.fit(df_train, y)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

### 3 seeds, 10 folds

In [ ]:
lucky_seeds=[42,2019,91373] # 늘려가면서
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True) # 늘려가면서
    cv = np.zeros((train_x.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

        rfmodel = RandomForestClassifier(n_estimators=1200, criterion='entropy', max_depth=60,
                                         min_samples_leaf=2, min_samples_split=2,
                                         random_state=seed)
        rfmodel.fit(x_train, y_train)
        #joblib.dump(rfmodel, f'./pred_pkl/RF_{n+1}_fold_{seed}_seed_rf.pkl')
        
        cv[val_idx, :] = rfmodel.predict_proba(x_val)      
        print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
        pred_test += rfmodel.predict_proba(test_x) / 10 # CV 바꾸면 이 숫자도 똑같이 바꿔야함
        
    pred_dict['rf'+str(seed)] = cv
    pred_test_dict['rf'+str(seed)] = pred_test
    print('multi_logloss :', log_loss(train_y, cv))

rfmodels_path = os.listdir('./pred_pkl/')
rfmodels_list = [x for x in rfmodels_path if x.endswith("rf.pkl")]
assert len(rfmodels_list) == 15
rf_preds = np.zeros((test_x.shape[0], 3))

for m in rfmodels_list:
    rfmodel = joblib.load('./pred_pkl/'+m)
    rf_preds_proba = rfmodel.predict_proba(test_x)
    rf_preds += rf_preds_proba/15

## (4) Catboost (성능X)

lucky_seeds=[42,2019,91373]
for i, seed in enumerate(lucky_seeds):

    kfold = KFold(n_splits=5, random_state = seed, shuffle = True)
    cv = np.zeros((train.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        _train = Pool(x_train, label=y_train)
        _valid = Pool(x_val, label=y_val)

        catmodel =  CatBoostClassifier(loss_function='MultiClass', early_stopping_rounds=50, 
                                       random_state=seed, learning_rate=0.02, iterations=100000
                                       #task_type="GPU"
                                      )
        
        catmodel.fit(_train, eval_set=_valid, use_best_model=True, verbose=2000)
        #joblib.dump(rfmodel, f'./pred_pkl/RF_{n+1}_fold_{seed}_seed_rf.pkl')
        
        cv[val_idx, :] = catmodel.predict_proba(x_val)        
        pred_test += catmodel.predict_proba(test_x) / 5
        
    pred_dict['cat'+str(i+1)] = cv
    pred_test_dict['cat'+str(i+1)] = pred_test
    print('multi_logloss :', log_loss(true, cv))

## (4) Stacking (AutoLGB)

### 27features = 3seed(42, 2019, 91373) x 3model(lgb, xgb, rf) x 3class(0, 1, 2)

In [ ]:
new_pred_dict_1={}
new_pred_test_dict_1={}
for i in range(len(pred_dict)):
    if log_loss(train_y, list(pred_dict.values())[i])<0.68:
        new_pred_dict_1[list(pred_dict.keys())[i]]=list(pred_dict.values())[i]
        new_pred_test_dict_1[list(pred_test_dict.keys())[i]]=list(pred_test_dict.values())[i]

In [ ]:
new_pred_dict_2={}
new_pred_test_dict_2={}
for i in range(len(pred_dict)):
    if log_loss(train_y, list(pred_dict.values())[i])<0.687:
        new_pred_dict_2[list(pred_dict.keys())[i]]=list(pred_dict.values())[i]
        new_pred_test_dict_2[list(pred_test_dict.keys())[i]]=list(pred_test_dict.values())[i]

In [ ]:
with open('./pkl/new_pred_dict_1.pickle', 'wb') as fw:
    pickle.dump(new_pred_dict_1, fw)
    
with open('./pkl/new_pred_test_dict_1.pickle', 'wb') as fw:
    pickle.dump(new_pred_test_dict_1, fw)

In [ ]:
with open('./pkl/new_pred_dict_2.pickle', 'wb') as fw:
    pickle.dump(new_pred_dict_2, fw)
    
with open('./pkl/new_pred_test_dict_2.pickle', 'wb') as fw:
    pickle.dump(new_pred_test_dict_2, fw)

In [ ]:
with open('./pkl/new_pred_dict_1.pickle', 'rb') as fw:
    new_pred_dict_1 = pickle.load(fw)

with open('./pkl/new_pred_test_dict_2.pickle', 'rb') as fw:
    new_pred_test_dict_2 = pickle.load(fw)

In [9]:
with open('./pkl/new_pred_dict_2.pickle', 'rb') as fw:
    new_pred_dict_2 = pickle.load(fw)

with open('./pkl/new_pred_test_dict_2.pickle', 'rb') as fw:
    new_pred_test_dict_2 = pickle.load(fw)

In [10]:
X_train = pd.DataFrame(np.hstack([x for _, x in new_pred_dict_2.items()]))
X_test = pd.DataFrame(np.hstack([x for _, x in new_pred_test_dict_2.items()]))

pred = np.zeros((X_train.shape[0], 3), dtype=float)
pred_test = np.zeros((X_test.shape[0], 3), dtype=float)
#kfold = KFold(n_splits=5, random_state = seed, shuffle = True)
cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=42) # 이건 CV 너무 크게하면 안됨, 3~6까지 테스트해보면 좋을듯

for i_cv, (i_trn, i_val) in enumerate(cv.split(X_train, train_y)):
    if i_cv == 0:
        clf = AutoLGB(objective='multiclass', metric='multi_logloss', params={'num_class': 3}, 
                      feature_selection=False, n_est=10000)
        clf.tune(X_train.iloc[i_trn], train_y[i_trn])
        n_best = clf.n_best
        features = clf.features
        params = clf.params
        print(f'best iteration: {n_best}')
        print(f'selected features ({len(features)}): {features}')        
        print(params)
        clf.fit(X_train.iloc[i_trn], train_y[i_trn])
    else:
        train_data = lgb.Dataset(X_train[features].iloc[i_trn], label=train_y[i_trn])
        clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    pred[i_val] = clf.predict(X_train[features].iloc[i_val])
    pred_test += clf.predict(X_test[features]) / 7

100%|██████████████████████████████████████████████| 100/100 [00:33<00:00,  2.98trial/s, best loss: 0.6793218524233182]
best iteration: 254
selected features (18): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
{'bagging_freq': 1, 'verbosity': -1, 'seed': 42, 'num_threads': -1, 'feature_pre_filter': False, 'num_class': 3, 'objective': 'multiclass', 'metric': 'multi_logloss', 'boosting': 'gbdt', 'bagging_fraction': 0.7000000000000001, 'feature_fraction': 0.5, 'lambda_l1': 0, 'lambda_l2': 0.1, 'learning_rate': 0.01278719685972846, 'max_depth': 4, 'min_child_samples': 25, 'num_leaves': 31}


In [11]:
print(f'CV Log Loss: {log_loss(train_y, pred):.6f}')

CV Log Loss: 0.668255


# 결과 제출

In [12]:
submission = sample_submission.copy()
submission.iloc[:, 1:] = pred_test

In [13]:
submission.to_csv('submission/submission.csv', index=False)